In [684]:
import pandas as pd
import torch
import PIL.Image
from torchvision import transforms
import  numpy as np
import  torch
import torchvision
import torch.nn.functional as F
from torch import  nn
from torch import optim


# 数据集预处理

In [685]:

data_file = "data/lungdatasets.csv"

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('max_colwidth', 100)

In [686]:
# csv使用分号分割，seq指定分隔符为；
data = pd.read_csv(data_file,sep=';')

# 住院时长周数为值
outputs = data.iloc[:, 25]

# 定义特征
all_features = data.iloc[:, 1:25]
strlist = ['性别(1=男，2=女）','入院时临床分型（0=无症状感染者，1=轻型，2=普通型，3=重型，4=危重型）','出院时临床分型（0=无症状感染者，1=轻型，2=普通型，3=重型，4=危重型）','出院时临床转归（0=临床治愈，1=病毒转阴）','首诊时有无发热（0=否，1=是）','以前曾是否患过肺炎（0=否，1=是）','病灶位置是否右上（1=是，0=否）','病灶位置是否右中（1=是，0=否）','病灶位置是否右下（1=是，0=否）','病灶位置是否左上（1=是，0=否）','病灶位置是否左下（1=是，0=否）']
all_features[strlist] = all_features[strlist].astype(np.str)
# print(all_features.dtypes)


In [687]:
# for i in all_features['身高（cm）']:
#     if(i==0):
#         i = None
#         print(i)

#修改数值特征中的所有0值为NAN
zero_2_nan_Str = ['身高（cm）','体重(kg)','首次检查病灶体积总和（cm?）','首次检查全部病灶占全肺体积占比(%)','首次检查全部病灶的平均密度（HU）','末次检查病灶体积总和（cm?）','末次检查全部病灶占全肺体积占比(%)','末次检查全部病灶的平均密度（HU）']
all_features[zero_2_nan_Str] = all_features[zero_2_nan_Str].replace(0,np.nan)
# all_features

In [688]:
# 若无法获得测试数据，则可根据训练数据计算均值和标准差
numeric_features = all_features.dtypes[all_features.dtypes != 'object'].index

# print(numeric_features[1])
# print(all_features[numeric_features])

all_features[numeric_features] = all_features[numeric_features].apply(
    lambda x: (x - x.mean()) / (x.std()))
# 在标准化数据之后，所有均值消失，因此我们可以将缺失值设置为0
all_features[numeric_features] = all_features[numeric_features].fillna(0)

# “Dummy_na=True”将“na”（缺失值）视为有效的特征值，并为其创建指示符特征
all_features = pd.get_dummies(all_features, dummy_na=False)
# all_features


## 数据集输入为inputs 输出为outpus 此时均为Dataframe类型

In [689]:
import  openpyxl
# xlsx_outputpath='data/after_lungdataset.xlsx'
# csv_outputpath='data/after_lungdataset.csv'
# all_features.to_excel(xlsx_outputpath,index=True,header=True)
# all_features.to_csv(csv_outputpath,sep=',',index=False,header=False)
# inputs = all_features
# outputs = outputs
# data = pd.concat([inputs, outputs], sort=False, axis=1)

data = pd.read_csv('data/Data.csv',sep=';')

# 住院时长周数为值
y = data.iloc[:, 48]

# 定义特征
x = data.iloc[:, 1:48]


# 划分数据集

In [690]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(
    x,y, test_size=0.2, random_state=None)


In [691]:
#输入
x_train = torch.tensor(x_train.values)
x_test = torch.tensor(x_test.values)
#输出
y_train = torch.tensor(y_train.values)
y_test = torch.tensor(y_test.values)

data_dim = x_train.shape[1]
# target_dim = target.shape[1]
# print(data)
print(data.shape)
print(data_dim,x_train.shape)
x_train.unique()

(437, 49)
47 torch.Size([349, 47])


tensor([-15.9531,  -6.6425,  -6.1671,  ...,   8.2956,   9.2392,  14.1437],
       dtype=torch.float64)

## 定义Dataset

In [692]:
from torch.utils.data import Dataset


class LungDataset(Dataset):

    def __init__(self,inputs,outputs):
        self.inputs = inputs
        self.outputs = outputs

    def __getitem__(self, idx):
        vector = self.inputs[idx]
        label = self.outputs[idx]
        return vector,label

    def __len__(self):
        return len(self.outputs)

trainDataset = LungDataset(x_train,y_train)
testDataset = LungDataset(x_test,y_test)
x,y = trainDataset[0]
x.shape



torch.Size([47])

# 搭建网络

In [693]:
class MnistNet(nn.Module):
    def __init__(self):
        super(MnistNet,self).__init__()
        self.fc1 = nn.Linear(47*1,28)  #定义Linear的输入和输出的形状
        self.fc2 = nn.Linear(28,10)  #定义Linear的输入和输出的形状

    def forward(self,x):
        x = x.view(-1,47*1)  #对数据形状变形，-1表示该位置根据后面的形状自动调整
        x = self.fc1(x) #[batch_size,28]
        x = F.relu(x)  #[batch_size,28]
        x = self.fc2(x) #[batch_size,10]
        return F.log_softmax(x, dim=-1)

In [694]:

# train_dataloader = torch.utils.data.DataLoader(data,batch_size=64,shuffle=True)

# 准备迭代器

In [695]:
train_batch_size = 64
test_batch_size = 128
img_size = data_dim

def get_dataloader(train=True):

    #准备数据集，其中0.1307，0.3081为MNIST数据的均值和标准差，这样操作能够对其进行标准化
    #因为MNIST只有一个通道（黑白图片）,所以元组中只有一个值
    dataset = trainDataset if train else testDataset
    #准备数据迭代器
    batch_size = train_batch_size if train else test_batch_size
    dataloader = torch.utils.data.DataLoader(dataset,batch_size=batch_size,shuffle=True)
    return dataloader

# 训练

In [696]:
mnist_net = MnistNet().double()
optimizer = optim.Adam(mnist_net.parameters(),lr= 0.001)
train_loss_list = []
train_count_list = []
def train(epoch):
    mnist_net.train(True)
    train_dataloader = get_dataloader(True)
    print("开始训练：")
    for idx,(data,target) in enumerate(train_dataloader):
        optimizer.zero_grad()
        output = mnist_net(data)
        loss = F.nll_loss(output,target) #对数似然损失
        loss.backward()
        optimizer.step()
        if idx % 200 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch,  idx * len(data), len(train_dataloader.dataset),100. * idx / len(train_dataloader), loss.item()))
            train_loss_list.append(loss.item())
            train_count_list.append(idx*train_batch_size+(epoch-1)*len(train_dataloader))
    print("结束训练。")

In [697]:
epoch = 300
for i in range(epoch):
    train(i)

开始训练：
Train Epoch: 0 [0/349 (0%)]	Loss: 2.343100
结束训练。
开始训练：
Train Epoch: 1 [0/349 (0%)]	Loss: 2.298818
结束训练。
开始训练：
Train Epoch: 2 [0/349 (0%)]	Loss: 2.255785
结束训练。
开始训练：
Train Epoch: 3 [0/349 (0%)]	Loss: 2.182099
结束训练。
开始训练：
Train Epoch: 4 [0/349 (0%)]	Loss: 2.161822
结束训练。
开始训练：
Train Epoch: 5 [0/349 (0%)]	Loss: 2.165835
结束训练。
开始训练：
Train Epoch: 6 [0/349 (0%)]	Loss: 2.148851
结束训练。
开始训练：
Train Epoch: 7 [0/349 (0%)]	Loss: 2.044852
结束训练。
开始训练：
Train Epoch: 8 [0/349 (0%)]	Loss: 1.947189
结束训练。
开始训练：
Train Epoch: 9 [0/349 (0%)]	Loss: 1.909846
结束训练。
开始训练：
Train Epoch: 10 [0/349 (0%)]	Loss: 1.929374
结束训练。
开始训练：
Train Epoch: 11 [0/349 (0%)]	Loss: 1.894190
结束训练。
开始训练：
Train Epoch: 12 [0/349 (0%)]	Loss: 1.861616
结束训练。
开始训练：
Train Epoch: 13 [0/349 (0%)]	Loss: 1.778987
结束训练。
开始训练：
Train Epoch: 14 [0/349 (0%)]	Loss: 1.834772
结束训练。
开始训练：
Train Epoch: 15 [0/349 (0%)]	Loss: 1.806285
结束训练。
开始训练：
Train Epoch: 16 [0/349 (0%)]	Loss: 1.902036
结束训练。
开始训练：
Train Epoch: 17 [0/349 (0%)]	Loss: 1.921909
结束训练。
开始

# 测试

In [698]:
def test():
    test_loss = 0
    correct = 0
    mnist_net.eval()
    test_dataloader = get_dataloader(train=False)
    with torch.no_grad():
        for data, target in test_dataloader:
            output = mnist_net(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            pred = output.data.max(1, keepdim=True)[1] #获取最大值的位置,[batch_size,1]
            correct += pred.eq(target.data.view_as(pred)).sum()
    test_loss /= len(test_dataloader.dataset)
    print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_dataloader.dataset),
        100. * correct / len(test_dataloader.dataset)))

In [699]:
test()


Test set: Avg. loss: 2.7306, Accuracy: 15/88 (17.05%)



In [700]:
torch.save(mnist_net.state_dict(),"model/mnist_net.pt") #保存模型参数
torch.save(optimizer.state_dict(), 'results/mnist_optimizer.pt') #保存优化器参数

FileNotFoundError: [Errno 2] No such file or directory: 'model/mnist_net.pt'

In [ ]:
mnist_net.load_state_dict(torch.load("model/mnist_net.pt"))
optimizer.load_state_dict(torch.load("results/mnist_optimizer.pt"))